# Preparation

## Import packages

In [1]:
# Inputs – from Nuo
import pickle
import torch
from torch import nn
import numpy as np
from torch.nn import Module

## Load traning and testing datasets

### Full-length transcripts

In [87]:
with open("proc/training_dataset.pkl", "rb") as f:
    training_dataset = pickle.load(f)

with open("proc/test_dataset.pkl", "rb") as f:
    test_dataset = pickle.load(f)

### 80nt transcripts

In [2]:
with open("proc/training_dataset_80nt.pkl", "rb") as f:
    training_dataset = pickle.load(f)

with open("proc/test_dataset_80nt.pkl", "rb") as f:
    test_dataset = pickle.load(f)

### Define NtDataset

In [7]:
# Dataset – from Hannah and Lucas

class NtDataset:
    """Nucleotide sequence + splice sites dataset."""
    def __init__(self, dataset):
        self.dataset = dataset
        self.map = {'A':0, 'G':1, 'C':2, 'T':3}
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        nt_seq = self.dataset[idx][0]
        strengths = self.dataset[idx][1]
        
        tokenized_seq = []
        
        for letter in nt_seq:
            tokenized_seq.append(self.map[letter])
            
        return torch.tensor(tokenized_seq), torch.tensor(strengths).type(torch.LongTensor)

## Model

In [8]:
# from torch.optim import optim

class SpliceFormer(nn.Module):
    """Transformer for splice site prediction"""

    def __init__(
        self,
        vocab_size:int,
        model_dim: int,
        n_attn_heads: int,
        n_encoder_layers: int,
        hidden_act: Module,
        dropout: float,
    ) -> None:

        super().__init__()

        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=n_attn_heads,
            dim_feedforward=model_dim,
            dropout=dropout,
            activation=hidden_act,
            batch_first=True)

        self.embedding = nn.Embedding(
            vocab_size, embedding_dim=model_dim)
        
        self.vocab_size = vocab_size

        self.encoder = nn.TransformerEncoder(
            encoder_layer=self.encoder_layer, num_layers=n_encoder_layers)

        self.out_layer = nn.Linear(in_features=model_dim, out_features=3, bias=False)
        # self.lm_head.weight = self.embedding.weight

    def forward(self, inputs):
        #self.vocab_size,
        x_emb = self.embedding(inputs)

        # inputs: (batch_size, seq_len, n_tokens)
        encoder_output = self.encoder(x_emb)
        outputs = self.out_layer(encoder_output)

        return outputs

# Training

In [11]:
training_dataset_subset = training_dataset

In [12]:
# Training

n_epochs = 40
nucleotide_loader = NtDataset(training_dataset_subset)
loss_fn = nn.CrossEntropyLoss()
splice_model = SpliceFormer(vocab_size=4, 
                            model_dim=64, 
                            n_attn_heads=2, 
                            n_encoder_layers=2, 
                            hidden_act=nn.ReLU(), 
                            dropout=0.1)

optimizer = torch.optim.SGD(splice_model.parameters(), lr=0.0001, momentum=0.9)

# training loop 
for epoch in range(n_epochs):

    running_loss = 0.0
    for seq_number, nucleotide_seq in enumerate(nucleotide_loader):
        inputs, labels = nucleotide_seq

        optimizer.zero_grad()
        
        # error is here!
        outputs = splice_model(inputs)
        # print(outputs)
        # print(labels)

        total_loss = loss_fn(outputs, labels)
        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()
        if seq_number % 1000 == 0:
            print(f'epoch: {epoch}, step: {seq_number}, loss: {running_loss}')
            torch.save(splice_model.state_dict(), f'./tbh_model_{seq_number}.pth')
            # running_loss = 0.0

            
print("Finished training!\nFinal loss value:", total_loss)
torch.save(splice_model.state_dict(), './tbh_model_final.pth')

epoch: 0, step: 0, loss: 0.9895215034484863
epoch: 0, step: 10, loss: 9.94553828239441
epoch: 0, step: 20, loss: 17.673371195793152
epoch: 0, step: 30, loss: 24.22443813085556
epoch: 0, step: 40, loss: 29.779487371444702
epoch: 0, step: 50, loss: 34.64477500319481
epoch: 0, step: 60, loss: 38.85314616560936
epoch: 0, step: 70, loss: 42.7998731136322
epoch: 0, step: 80, loss: 45.94551703333855
epoch: 0, step: 90, loss: 48.873563677072525
epoch: 0, step: 100, loss: 51.956222623586655
epoch: 0, step: 110, loss: 54.78481739759445
epoch: 0, step: 120, loss: 57.238715410232544
epoch: 0, step: 130, loss: 59.47136886417866
epoch: 0, step: 140, loss: 61.52211035788059
epoch: 0, step: 150, loss: 63.50655588507652
epoch: 0, step: 160, loss: 65.36693830788136
epoch: 0, step: 170, loss: 67.20206490159035
epoch: 0, step: 180, loss: 68.91383616626263
epoch: 0, step: 190, loss: 70.85664342343807
epoch: 0, step: 200, loss: 72.75173152983189
epoch: 0, step: 210, loss: 74.26308397948742
epoch: 0, step: 2

KeyboardInterrupt: 

In [67]:
for seq_number, nucleotide_seq in enumerate(nucleotide_loader):
    inputs, labels = nucleotide_seq
    labels = labels.reshape(-1)
    # labels = labels.unsqueeze(0)
    break

outputs = splice_model(inputs)
labels = labels.type(torch.LongTensor)
# outputs = outputs.unsqueeze(0)
print(outputs.shape, labels.shape)
total_loss = loss_fn(outputs, labels)
print(total_loss)
# print(outputs.shape, labels.shape)

torch.Size([80, 3]) torch.Size([80])
tensor(1.2545, grad_fn=<NllLossBackward0>)


In [ ]:
# Test

nucleotide_test_loader = NtDataset(test_dataset)
splice_test = SpliceFormer()
splice_test.load_state_dict(torch.load('./tbh_model_final.pth', weights_only = True)) 

outputs = splice_test.(nucleotide_seq)

